In [0]:
dbutils.fs.mount(
  source = "wasbs://pacchi@pacchi.blob.core.windows.net",
  mount_point = "/mnt/retail_db",
  extra_configs = {'fs.azure.account.key.pacchi.blob.core.windows.net' :'P/66EoKeJ6xmvsiqBmu/ic4SaU8+8Z59hBv03Il+01zY+H4KR0g11WHA22A6Mo19rYhO7EDdbSw3+AStoFgzKg=='}
)

In [0]:
%sql
create database tripdb

In [0]:
%fs
ls dbfs:/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2009-01.csv.gz

In [0]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2009-01.csv.gz")


In [0]:
display(df)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS tripsdb

In [0]:
df.repartition(20).write.mode('overwrite').format('delta').option('path','/mnt/tripsdb/trips_delta').saveAsTable("tripsdb.trips_delta")

In [0]:
%sql
describe detail tripsdb.trips_delta;

In [0]:
%sql
select count(1) from tripsdb.trips_delta

In [0]:
df.repartition(20).write.mode('overwrite').format('parquet').option('path','/mnt/tripdb/trips_parquet').saveAsTable("tripsdb.trips_parquet")

In [0]:
%sql
describe extended tripsdb.trips_parquet;

In [0]:
%sql
select count(*) from tripsdb.trips_delta;

In [0]:
%sql
select count(*) from tripsdb.trips_parquet;

In [0]:
spark.conf.get("spark.databricks.io.cache.enabled")

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "true")

In [0]:
%sql
select sum(Trip_Distance), sum(Total_Amt) from tripsdb.trips_delta;

In [0]:
%sql
select sum(Trip_Distance), sum(Total_Amt) from tripsdb.trips_delta;

In [0]:
%sql
UNCACHE TABLE tripsdb.trips_delta 

In [0]:
%sql 
uncache table tripsdb.trips_delta; select sum(Trip_Distance), sum(Total_Amt) from tripsdb.trips_delta;

In [0]:
display(df)

In [0]:
df.repartition(500).write.mode('overwrite').format('delta').option('path','/mnt/tripsdb/bin/trips_delta').saveAsTable("tripsdb.trips_delta_bin")

In [0]:
%sql
select * from tripsdb.trips_delta_bin where Total_Amt = 20;

In [0]:
%sql
select * from tripsdb.trips_delta where Total_Amt = 20; 

In [0]:
%sql
optimize tripsdb.trips_delta_bin

In [0]:
%sql
select * from tripsdb.trips_delta_bin where Total_Amt = 20;

In [0]:
df.repartition(200).write.mode('append').format('delta').saveAsTable("tripsdb.trips_delta_new")

In [0]:
%sql
describe detail tripsdb.trips_delta_new;

In [0]:
%sql
select * from tripsdb.trips_delta_new where Passenger_Count = 4;

In [0]:
%sql
optimize tripsdb.trips_delta_new zorder by (Passenger_Count)

In [0]:
%sql
select * from tripsdb.trips_delta_new where Passenger_Count = 4;

In [0]:
%sql
select * from tripsdb.trips_delta_new where Passenger_Count = 4;

In [0]:
%sql
describe detail tripsdb.trips_delta;

In [0]:
%sql
vacuum tripsdb.trips_delta retain 1 hours dry run

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [0]:
%sql
vacuum tripsdb.trips_delta retain 1 hours dry run

In [0]:
%sql Select * from tripsdb.trips_delta version as of 1;

In [0]:
%sql
describe detail tripsdb.trips_delta

In [0]:
%sql Select * from tripsdb.trips_delta version as of 0